# Executamos e testamos esse notebook apenas no colab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download e instalação do modelo Tortoise-TTS

In [3]:
# the scipy version packaged with colab is not tolerant of misformated WAV files.
# install the latest version.
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1481, done.
remote: Total 1481 (delta 0), reused 0 (delta 0), pack-reused 1481
Receiving objects: 100% (1481/1481), 53.56 MiB | 27.33 MiB/s, done.
Resolving deltas: 100% (604/604), done.
/content/tortoise-tts
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.1/819.1 kB 15.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━

In [4]:
# Importações utilizadas através do restante do notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# Isso fará o download de todos os modelos usados pela Tortoise a partir do hub HuggingFace.
tts = TextToSpeech()

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

...

Done.


100% (25193729 of 25193729) |############| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


100% (100715777 of 100715777) |##########| Elapsed Time: 0:00:01 Time:  0:00:01


Done.


# Fazendo o upload dos audios que serão clonados

In [5]:
# Execute a celula se você for fazer o upload de uma voz dos seus downloads
# Recomendação pelo menos 2 áudio em formato arquivo WAV, com 6 a 10 segundos de duração.
CUSTOM_VOICE_NAME = "martin"

import os
from google.colab import files

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder,  exist_ok=True)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

Saving Miley _cortes3.wav to Miley _cortes3.wav
Saving Miley_cortes1.wav to Miley_cortes1.wav
Saving Miley_cortes2.wav to Miley_cortes2.wav


### Gerar fala mais longa a partir de um arquivo de texto

In [7]:
from tortoise.utils.text import split_and_recombine_text
from time import time
import os

outpath = "results/longform/"

# Escreva um texto em inglês, preferencialmente
text = """ We were good, we were gold, Kind of dream that can't be sold, We were right 'til we weren't,Built a home and watched it burn"""

# Verifica se o texto contém o caractere '|'
if '|' in text:
    print("Found the '|' character in your text, which I will use as a cue for where to split it up. If this was not"
          "your intent, please remove all '|' characters from the input.")
    texts = text.split('|')
else:
    texts = split_and_recombine_text(text)

seed = int(time())

voice_outpath = os.path.join(outpath, CUSTOM_VOICE_NAME)
os.makedirs(voice_outpath, exist_ok=True)

voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)

all_parts = []
for j, text in enumerate(texts):
    gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                              preset="fast", k=1, use_deterministic_seed=seed)
    gen = gen.squeeze(0).cpu()
    torchaudio.save(os.path.join(voice_outpath, f'{j}.wav'), gen, 24000)
    all_parts.append(gen)

full_audio = torch.cat(all_parts, dim=-1)
torchaudio.save(os.path.join(voice_outpath, 'combined.wav'), full_audio, 24000)
IPython.display.Audio(os.path.join(voice_outpath, 'combined.wav'))


/content/tortoise-tts/tortoise/utils/audio.py:14: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sampling_rate, data = read(full_path)


Generating autoregressive samples..


100%|██████████| 6/6 [00:34<00:00,  5.74s/it]


Computing best candidates using CLVP and CVVP


100%|██████████| 6/6 [00:09<00:00,  1.54s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:17<00:00,  4.68it/s]
